In [ ]:
!make

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from clawpack import pyclaw
from clawpack import riemann
import reactive_euler_1D_roe_efix as rp_solver
import steadyState as steadyState

In [ ]:
%matplotlib inline
gamma = 1.2
gamma1 = gamma - 1
qheat = 50
Ea = 26
T_ign = 4
mx = 200
xmax=25
tfinal = 100
dtplot = 1

In [ ]:
def step_reaction(solver, state, dt):
    global k
    q = state.q
    q[0,:] = q[0,:]
    q[1,:] = q[1,:]
    q[2,:] = q[2,:]
    q[3,:] = q[3,:] + dt*omega(q, k)

def step_reaction_dq(solver, state, dt):
    global k
    q = state.q
    dq = np.empty(q.shape)
    dq[3,:] = dt*omega(q, k)
    return dq
    
def omega(q, k):
    """ Reaction rate function
    """
    import numpy as np
    pressure = gamma1*(q[2,:]-0.5*q[1,:]**2/q[0,:] -
                        qheat*q[3,:])
    T = pressure / q[0,:]
    return -k*(q[3,:])*np.exp(-Ea/T)*(T>T_ign)

In [ ]:
solver = pyclaw.ClawSolver1D(rp_solver)
solver.step_source = step_reaction
solver.limiters = 1
#solver = pyclaw.SharpClawSolver1D(reactive_euler_efix_roe_1D)
#solver.dq_src = step_reaction_dq
#solver.weno_order = 5
solver.bc_lower[0]=pyclaw.BC.extrap
solver.bc_upper[0]=pyclaw.BC.extrap
solver.num_eqn = 4
solver.num_waves = 4

In [ ]:
 # Initialize domain
x = pyclaw.Dimension(0.0,xmax,mx,'x')
domain = pyclaw.Domain([x])
state = pyclaw.State(domain,solver.num_eqn)

x =state.grid.x.centers
xs = xmax-5
xdet = x[x<xs]

rhol, Ul, pl, laml, D, k = steadyState.steadyState(qheat, Ea, gamma, xdet)
ul = Ul + D
Yl = 1-laml
print("k="+str(k))

rhor = 1.*np.ones(np.shape(x[x>=xs]))
ur = 0.0*np.ones(np.shape(x[x>=xs]))
pr = 1*np.ones(np.shape(x[x>=xs]))
Yr = 1.*np.ones(np.shape(x[x>=xs]))

rho = np.append(rhol, rhor)
u = np.append(ul, ur)
p = np.append(pl, pr)
Y = np.append(Yl, Yr)

In [ ]:
state.q[0,:] = rho
state.q[1,:] = rho*u
state.q[2,:] = p/gamma1 + rho*u**2/2 + qheat*rho*Y
state.q[3,:] = rho*Y   

state.problem_data['gamma']= gamma
state.problem_data['gamma1']= gamma1
state.problem_data['qheat']= qheat
state.problem_data['Ea'] = Ea
state.problem_data['T_ign'] = T_ign
state.problem_data['xfspeed'] = 1.*D

In [ ]:
claw = pyclaw.Controller()
claw.tfinal = tfinal
claw.keep_copy = True       # Keep solution data in memory for plotting
claw.num_output_times = np.ceil(claw.tfinal / dtplot)  # Write 50 output frames
claw.solution = pyclaw.Solution(state,domain)
claw.solver = solver

Finally, let's run the simulation.

In [ ]:
status = claw.run();

## Plotting
Now we'll plot the results, which are contained in `claw.frames[:]`.  It's simple to plot a single frame with matplotlib:

In [ ]:
def pressure(current_data):
    q = current_data.q
    rho = q[0,:]
    u   = q[1,:]/rho
    press = gamma1 *  (q[2,:] - 0.5*rho*(u**2) - qheat*q[3,:])
    return press

In [ ]:
frameno=0
density = claw.frames[frameno].q[0,:]
p = pressure(claw.frames[frameno])
plt.plot(x,p,'-')
plt.title("time="+str(claw.frames[frameno].t))

To examine the evolution more thoroughly, it's nice to see all the frames in sequence.  We can do this as follows.

In [ ]:
from matplotlib import animation
from clawpack.visclaw.JSAnimation import IPython_display

frame = claw.frames[0]
density = frame.q[0,:]
p = pressure(frame)

fig = plt.figure(figsize=(4,2))
ax = plt.axes(xlim=(0, xmax), ylim=(0, np.max(p)+10))
line, = ax.plot([], [], lw=2)
plt.title("time="+str(frame.t))


def fplot(frame_number):
    frame = claw.frames[frame_number]
    p = pressure(frame)
    t = frame.t
    xlab = x+D*t
    ax.set_xlim([xlab[0], xlab[-1]])
    line.set_data(x+D*t,p)
    ax.set_title('Pressure at t='+str(t))
    return line,

animation.FuncAnimation(fig, fplot, frames=len(claw.frames), interval=50, blit=True)

In [ ]:
import matplotlib
matplotlib.use("Agg")

FFMpegWriter = animation.writers['ffmpeg']
metadata = dict(title='Movie Test', artist='Matplotlib',
        comment='Movie support!')
writer = FFMpegWriter(fps=15, metadata=metadata)

frame = claw.frames[0]
density = frame.q[0,:]
p = pressure(frame)

fig = plt.figure(figsize(4,2))
ax = plt.axes(xlim=(0, xmax), ylim=(0, np.max(p)+10))
line, = ax.plot([], [], lw=2)
plt.title("time="+str(frame.t))

with writer.saving(fig, "pulsating_detonation.mp4", 100):
    for i in range(len(claw.frames)):
        fplot(i)
        writer.grab_frame()